In [1]:
USER_ID = 'kevinbot'
USER_SECRET = ''

In [2]:
import os, json, nbformat
os.makedirs('.kaggle', exist_ok = True)
with open(os.path.join('.kaggle', 'kaggle.json'), 'w') as f:
    json.dump({'username': USER_ID, 'key': USER_SECRET}, f)
!chmod 600 .kaggle/kaggle.json

In [3]:
notebook_meta_template = lambda user_id, title, file_id, nb_path: {'id': f'{user_id}/{file_id}',
 'title': f'{title}',
 'code_file': nb_path,
 'language': 'python',
 'kernel_type': 'notebook',
 'is_private': True,
 'enable_gpu': True,
 'enable_internet': False,
 'keywords': [],
 'dataset_sources': ['gaborfodor/keras-pretrained-models', 'kmader/food41'],
 'kernel_sources': [],
 'competition_sources': []}

In [4]:
base_dir = os.path.join('.', 'base_kernel')
os.makedirs(base_dir, exist_ok = True)

In [5]:
kernel_path = os.path.join(base_dir ,'hot-dog-not-hot-dog-gpu.ipynb')
if not os.path.exists(kernel_path):
    !kaggle kernels pull -k kmader/hot-dog-not-hot-dog-gpu -p {base_dir}

In [6]:
import keras
model_list = [('vgg16', 'VGG16'),
              ('inception_v3', 'InceptionV3'),
              ('xception', 'Xception')
             ]
make_model = lambda mod_folder, mod_name: 'from keras.applications.{0} import {1} as PTModel, preprocess_input'.format(mod_folder, mod_name)
search_text = '##### INSERT_MODEL_IMPORT #######'

Using TensorFlow backend.


In [ ]:
launched_kernels_list = []
cur_nb = nbformat.read(kernel_path, as_version = 4)
for i, (mod_folder, mod_name) in enumerate(model_list):
    out_name = 'test_{:04d}'.format(i)
    out_kernel_path = '{}.ipynb'.format(out_name)
    with open(kernel_path, 'r') as fi:
        all_lines = fi.readlines()
        with open(out_kernel_path, 'w') as fo:
            new_lines = [c_line.replace(search_text, 
                    make_model(mod_folder, mod_name)) for c_line in all_lines]
            fo.writelines(new_lines)
    with open('kernel-metadata.json', 'w') as f:
        meta_dict = notebook_meta_template(USER_ID, 
                           out_name, 
                           out_name, 
                           out_kernel_path)
        json.dump(meta_dict, f)
    out_str = !kaggle kernels push -p .
    launched_kernels_list += [out_str] 
    break

In [28]:
!kaggle kernels status -k kevinbot/test-0000

kevinbot/test-0000 has status "running"
